In [3]:
import gurobipy as gp
from gurobipy import GRB

In [4]:
def my_callback(model, where):
    if where == GRB.Callback.MIPSOL:
        # Feasible integer solution found
        try:
            solution = model.cbGetSolution(model.getVars())
            print("Feasible solution found:", solution)
            model._feasible_solutions.append(solution)
        except gp.GurobiError as e:
            print(f"Error retrieving feasible solution: {e}")
    
    elif where == GRB.Callback.MIPNODE:
        try:
            # Check node status
            node_status = model.cbGet(GRB.Callback.MIPNODE_STATUS)
            
            if node_status == GRB.Status.OPTIMAL:
                # Optimal relaxation available
                relaxation_solution = model.cbGetNodeRel(model.getVars())
                #is_feasible = test_relaxation_feasibility(model, relaxation_solution)
                # print(f"Optimal relaxation solution is {is_feasible} and the solution is: {relaxation_solution}")
                # model._relaxation_solutions.append({"status": "Optimal",  "feasiable": is_feasible, "solution": relaxation_solution})
                
                print(f"Optimal relaxation solution is: {relaxation_solution}")
                model._relaxation_solutions.append({"status": "Optimal", "solution": relaxation_solution})
            
            elif node_status == GRB.Status.INFEASIBLE:
                # Node relaxation is infeasible
                # print("Infeasible relaxation at node.")
                model._relaxation_solutions.append({"status": "Infeasible", "solution": None})
            
            elif node_status == GRB.Status.UNBOUNDED:
                # Node relaxation is unbounded
                # print("Unbounded relaxation at node.")
                model._relaxation_solutions.append({"status": "Unbounded", "solution": None})
            
            elif node_status == GRB.Status.INF_OR_UNBD:
                # Node relaxation is infeasible or unbounded
                # print("Infeasible or unbounded relaxation at node.")
                model._relaxation_solutions.append({"status": "InfOrUnbd", "solution": None})
            
            else:
                # print(f"Unhandled node status: {node_status}")
                model._relaxation_solutions.append({"status": "Unknown", "solution": None})
        
        except gp.GurobiError as e:
            pass
            # print(f"Error retrieving relaxation solution: {e}")

# Read the problem
number = "0031"
grb_model = gp.read(f"QPLIB_{number}.lp")

# Solution storage
grb_model._feasible_solutions = []
grb_model._relaxation_solutions = []

#model.setParam("NodeLimit", 100)  # Explore a limited number of nodes

# Optimize
grb_model.optimize(my_callback)

Restricted license - for non-production use only - expires 2025-11-24
Read LP format model from file QPLIB_0031.lp
Reading time = 0.00 seconds
obj: 32 rows, 60 columns, 120 nonzeros
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD Ryzen Threadripper PRO 5955WX 16-Cores, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 32 rows, 60 columns and 120 nonzeros
Model fingerprint: 0x00d24133
Model has 464 quadratic objective terms
Variable types: 30 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [3e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Feasible solution found: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [5]:
# Print solutions
print("\nCollected feasible solutions:")
for idx, sol in enumerate(grb_model._feasible_solutions):
    print(f"Solution {idx + 1}: {sol}")


Collected feasible solutions:
Solution 1: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
Solution 2: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Solution 3: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Solution 4: [0.0, 0.0, 0.0, 0.

In [6]:
# print("\nCollected relaxation solutions:")
# for idx, sol in enumerate(model._relaxation_solutions):
#     print(f"Relaxation {idx + 1} ({sol['status']}): {sol['solution']}")

In [7]:
import re
import numpy as np

# Initialize variables
objective_section = False
constraints_section = False
binary_section = False

objective_lines = []
constraint_lines = []
binary_lines = []

# Read the file
with open(f'QPLIB_{number}.txt', 'r') as f:
    for line in f:
        line = line.strip()
        if line.startswith('Minimize'):
            objective_section = True
            constraints_section = False
            binary_section = False
            continue
        elif line.startswith('Subject To'):
            objective_section = False
            constraints_section = True
            binary_section = False
            continue
        elif line.startswith('Binary'):
            objective_section = False
            constraints_section = False
            binary_section = True
            continue
        elif line.startswith('End'):
            objective_section = False
            constraints_section = False
            binary_section = False
            continue

        if objective_section:
            objective_lines.append(line)
        elif constraints_section:
            constraint_lines.append(line)
        elif binary_section:
            binary_lines.append(line)

# Collect variables
variables_set = set()

# From the 'Binary' section
binary_text = ' '.join(binary_lines)
binary_vars = re.findall(r'\b[b,x]\d+\b', binary_text)
variables_set.update(binary_vars)

# From the 'Subject To' section
constraint_text = ' '.join(constraint_lines)
vars_in_constraints = re.findall(r'\b[b,x]\d+\b', constraint_text)
variables_set.update(vars_in_constraints)

# From the objective function
objective_text = ' '.join(objective_lines)
vars_in_objective = re.findall(r'\b[b,x]\d+\b', objective_text)
variables_set.update(vars_in_objective)

# Function to extract variable type and number
def extract_var_num(var_name):
    m = re.match(r'([bx])(\d+)', var_name)
    if m:
        var_type = m.group(1)
        var_num = int(m.group(2))
        return var_type, var_num
    else:
        return None, None

# Create list of variables with type and number
variables_info = []
for var in variables_set:
    var_type, var_num = extract_var_num(var)
    if var_type is not None:
        variables_info.append((var_type, var_num, var))

# Sort variables: 'x' variables first, then 'b' variables, both sorted by number
variables_info.sort(key=lambda x: (0 if x[0]=='x' else 1, x[1]))

# Create variable_indices mapping
variable_indices = {var_info[2]: idx for idx, var_info in enumerate(variables_info)}

n = len(variable_indices)

# Now, parse the objective function
# Extract the quadratic terms inside the square brackets
quadratic_match = re.search(r'\[(.*)\]/2', objective_text)
if quadratic_match:
    quadratic_text = quadratic_match.group(1)
else:
    print("No quadratic terms found in the objective function.")
    quadratic_text = ''

# Replace '-' with '+ -' to split terms correctly
quadratic_text = quadratic_text.replace('-', '+ -')
terms = quadratic_text.split('+')

Q = np.zeros((n, n))

# Parse quadratic terms and build Q matrix
for term in terms:
    term = term.strip()
    if term == '':
        continue
    # Match terms like '52.8828 x2^2' or '-63.7552 x2 * x3'
    # First, split the term to separate the coefficient
    match = re.match(r'([+-]?\s*\d*\.?\d+(?:[eE][+-]?\d+)?)\s*(.*)', term)
    if match:
        coeff_str = match.group(1).replace(' ', '')
        coeff = float(coeff_str)
        vars_part = match.group(2)
        vars_part = vars_part.strip()
        # Check for squared terms
        squared_match = re.match(r'([bx]\d+)\^2', vars_part)
        if squared_match:
            var_name = squared_match.group(1)
            if var_name in variable_indices:
                i = variable_indices[var_name]
                # Update Q[i, i]
                Q[i, i] += coeff
            else:
                print(f"Variable {var_name} not found.")
        else:
            # Check for cross terms
            cross_match = re.match(r'([bx]\d+)\s*\*\s*([bx]\d+)', vars_part)
            if not cross_match:
                # Try matching 'x2*x3' without spaces
                cross_match = re.match(r'([bx]\d+)\*(?:\s*)([bx]\d+)', vars_part)
            if cross_match:
                var1_name = cross_match.group(1)
                var2_name = cross_match.group(2)
                if var1_name in variable_indices and var2_name in variable_indices:
                    i = variable_indices[var1_name]
                    j = variable_indices[var2_name]
                    # Update Q[i, j] and Q[j, i]
                    Q[i, j] += coeff
                    Q[j, i] += coeff
                else:
                    print(f"Variables {var1_name} or {var2_name} not found.")
            else:
                print(f"Could not parse term: {term}")
    else:
        print(f"Could not parse term: {term}")

# Now process the constraints
constraints_eq = []
constraints_leq = []
constraints_geq = []

for line in constraint_lines:
    line = line.strip()
    # Skip empty lines or comments
    if not line or line.startswith('\\'):
        continue
    # Handle constraints that may span multiple lines
    while not re.search(r'(=|<=|>=)\s*[\d\.]+$', line) and constraint_lines:
        next_line = constraint_lines.pop(0).strip()
        line += ' ' + next_line
    # Remove constraint name if any
    if ':' in line:
        _, rest = line.split(':', 1)
        rest = rest.strip()
    else:
        rest = line
    # Split on operators
    operator_match = re.match(r'(.+?)(<=|>=|=)(.+)', rest)
    if operator_match:
        lhs = operator_match.group(1).strip()
        operator = operator_match.group(2)
        rhs = operator_match.group(3).strip()
    else:
        print(f"Could not parse constraint line: {line}")
        continue
    # Parse the LHS to extract variables
    # Replace '-' with '+ -' to split terms correctly
    lhs = lhs.replace('-', '+ -')
    terms = lhs.split('+')
    coeffs = {}
    for term in terms:
        term = term.strip()
        if term == '':
            continue
        # Match coefficient and variable
        term_match = re.match(r'([+-]?\s*\d*\.?\d*)\s*([bx]\d+)', term)
        if term_match:
            coeff_str = term_match.group(1).replace(' ', '')
            if coeff_str in ['', '+']:
                coeff = 1.0
            elif coeff_str == '-':
                coeff = -1.0
            else:
                coeff = float(coeff_str)
            var_name = term_match.group(2)
        else:
            # Assume coefficient of 1
            coeff = 1.0
            var_name = term
            if var_name.startswith('-'):
                coeff = -1.0
                var_name = var_name[1:].strip()
        if var_name in variable_indices:
            idx = variable_indices[var_name]
            coeffs[idx] = coeffs.get(idx, 0) + coeff
        else:
            print(f"Variable {var_name} not found in constraints.")
    # Get RHS value
    rhs_value = float(rhs)
    # Store the coefficients and RHS value
    if operator == '=':
        constraints_eq.append((coeffs, rhs_value))
    elif operator == '<=':
        constraints_leq.append((coeffs, rhs_value))
    elif operator == '>=':
        constraints_geq.append((coeffs, rhs_value))
    else:
        print(f"Unknown operator {operator} in constraint.")
        continue

# Build E matrix and d vector for equality constraints
m_eq = len(constraints_eq)
E = np.zeros((m_eq, n))
d = np.zeros(m_eq)

for row_idx, (coeffs, rhs_value) in enumerate(constraints_eq):
    for idx, coeff in coeffs.items():
        E[row_idx, idx] = coeff
    d[row_idx] = rhs_value

# Build A matrix and b vector for inequality constraints (<=)
m_leq = len(constraints_leq)
A_leq = np.zeros((m_leq, n))
b_leq = np.zeros(m_leq)

for row_idx, (coeffs, rhs_value) in enumerate(constraints_leq):
    for idx, coeff in coeffs.items():
        A_leq[row_idx, idx] = coeff
    b_leq[row_idx] = rhs_value

# Build G matrix and h vector for inequality constraints (>=), converting them to -A x <= -b
m_geq = len(constraints_geq)
A_geq = np.zeros((m_geq, n))
b_geq = np.zeros(m_geq)

for row_idx, (coeffs, rhs_value) in enumerate(constraints_geq):
    for idx, coeff in coeffs.items():
        A_geq[row_idx, idx] = -coeff
    b_geq[row_idx] = -rhs_value

# Combine inequality constraints
A = np.vstack([A_leq, A_geq])
b_vector = np.hstack([b_leq, b_geq])

# Extract binary variables
binary_vars = re.findall(r'\b[b,x]\d+\b', binary_text)
binary_indices = []
for var in binary_vars:
    if var in variable_indices:
        idx = variable_indices[var]
        binary_indices.append(idx)
    else:
        print(f"Binary variable {var} not found.")

# Output the variable indices mapping
print("Variable Indices:")
for var, idx in variable_indices.items():
    print(f"{var}: {idx}")
    

# Output the results
print("\nMatrix A (for Ax ≤ b):")
print(A)
print("\nVector b:")
print(b_vector)
print("\nMatrix E (for Ex = d):")
print(E)
print("\nVector d:")
print(d)
print("\nCost matrix Q:")
print(Q)
print("\nBinary variable indices:")
print(binary_indices)


Variable Indices:
x2: 0
x3: 1
x4: 2
x5: 3
x6: 4
x7: 5
x8: 6
x9: 7
x10: 8
x11: 9
x12: 10
x13: 11
x14: 12
x15: 13
x16: 14
x17: 15
x18: 16
x19: 17
x20: 18
x21: 19
x22: 20
x23: 21
x24: 22
x25: 23
x26: 24
x27: 25
x28: 26
x29: 27
x30: 28
x31: 29
b32: 30
b33: 31
b34: 32
b35: 33
b36: 34
b37: 35
b38: 36
b39: 37
b40: 38
b41: 39
b42: 40
b43: 41
b44: 42
b45: 43
b46: 44
b47: 45
b48: 46
b49: 47
b50: 48
b51: 49
b52: 50
b53: 51
b54: 52
b55: 53
b56: 54
b57: 55
b58: 56
b59: 57
b60: 58
b61: 59

Matrix A (for Ax ≤ b):
[[ 0.  0.  0. ...  1.  1.  1.]
 [ 1.  0.  0. ...  0.  0.  0.]
 [ 0.  1.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... -1.  0.  0.]
 [ 0.  0.  0. ...  0. -1.  0.]
 [ 0.  0.  0. ...  0.  0. -1.]]

Vector b:
[5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]

Matrix E (for Ex = d):
[[1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.

In [8]:
# x_combined = np.random.rand(15)  # Replace with actual variable values
# b = np.random.rand(10)  # Replace with actual RHS values

In [9]:
# x_combined

In [83]:
# len(conts_values)?

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader  # Corrected import
from torch_geometric.nn import MessagePassing
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import gurobipy as gp
from gurobipy import GRB
import random

# Replace these placeholders with your actual data
# A = ...         # Constraint matrix A (m x n)
# variables_info = [...]  # Variable types (list of 'x' or 'b')
# b_vector = ...  # RHS vector b (array of length m)

# Example data (for demonstration purposes; replace with your actual data)
# A = np.array([[1, 2], [3, 4]])
# variables_info = ['x', 'b']
# b_vector = np.array([10, 20])

# Ensure that A, variables_info, and b_vector are defined
m, n = A.shape

# Get indices of non-zero elements in A
row_indices, col_indices = np.nonzero(A)
edge_weights = A[row_indices, col_indices]

# Map variable types to numerical values
# For node types: 0 - continuous, 1 - binary, 2 - constraint node
variable_types = np.array([0 if v == 'x' else 1 for v in variables_info])

# Set up the Gurobi model
model = gp.Model("FeasibleSolutionGenerator")
model.Params.LogToConsole = 0  # Turn off Gurobi output

# Create variables
variables = []
for i, var_type in enumerate(variables_info):
    if var_type == 'x':
        # Continuous variable between lower and upper bounds (adjust as needed)
        var = model.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name=f"x_{i}")
    else:
        # Binary variable
        var = model.addVar(vtype=GRB.BINARY, name=f"x_{i}")
    variables.append(var)

model.update()

# Add constraints
for i in range(m):
    expr = gp.LinExpr()
    for j in range(n):
        coeff = A[i, j]
        if coeff != 0:
            expr += coeff * variables[j]
    model.addConstr(expr <= b_vector[i], name=f"c_{i}")

model.update()

# Set objective function (dummy objective since we only need feasible solutions)
model.setObjective(0, GRB.MINIMIZE)

# Set Gurobi parameters for the solution pool
model.setParam('PoolSearchMode', 2)       # Find multiple diverse solutions
model.setParam('PoolSolutions', 1000)     # Adjust the number as needed
model.setParam('PoolGap', 0.1)            # Accept solutions within 10% of optimality

# Optimize the model
model.optimize()

# Check if the model is feasible
if model.Status == GRB.OPTIMAL or model.Status == GRB.SUBOPTIMAL or model.Status == GRB.TIME_LIMIT:
    num_solutions = model.SolCount
    print(f"Number of solutions found: {num_solutions}")

    feasible_solutions = []

    # Retrieve solutions from the pool
    for i in range(num_solutions):
        model.setParam(GRB.Param.SolutionNumber, i)
        solution = model.getAttr('Xn', variables)
        feasible_solutions.append(np.array(solution))
else:
    print("No feasible solution found.")
    feasible_solutions = []

# Function to generate infeasible sample
def generate_infeasible_sample():
    variable_values = np.zeros(n)
    for i, var_type in enumerate(variables_info):
        if var_type == 'x':
            # Continuous variable outside the feasible range
            variable_values[i] = np.random.uniform(-100, 100)
        else:
            # Binary variable (0 or 1)
            variable_values[i] = np.random.randint(0, 2)
    # Ensure the sample is infeasible
    Ax = A @ variable_values
    if np.all(Ax <= b_vector):
        # If the sample is feasible, adjust it to make it infeasible
        variable_values += np.random.uniform(1, 10, size=n)
    return variable_values

# Prepare the dataset
num_feasible_samples = len(feasible_solutions)
num_infeasible_samples = num_feasible_samples  # For balance
data_list = []

print("Preparing feasible samples...")
for x_combined in tqdm(feasible_solutions):
    # Compute ReLU(Ax - b), which should be zero for feasible samples
    Ax = A @ x_combined
    Ax_minus_b = Ax - b_vector
    relu_Ax_minus_b = np.maximum(Ax_minus_b, 0)  # Shape: (m,)

    # Prepare node features and types
    variable_node_features = x_combined.reshape(-1, 1)  # Shape: (n, 1)
    variable_node_type = variable_types  # Shape: (n,)
    constraint_node_features = b_vector.reshape(-1, 1)  # Shape: (m, 1)
    constraint_node_type = np.full(m, 2)  # Constraint node type
    node_features = np.concatenate([variable_node_features, constraint_node_features], axis=0)  # Shape: (n + m, 1)
    node_types = np.concatenate([variable_node_type, constraint_node_type], axis=0)  # Shape: (n + m,)

    # Edge indices and attributes
    source_nodes = col_indices
    target_nodes = row_indices + n  # Shift constraint node indices
    edge_index = np.stack([source_nodes, target_nodes], axis=0)
    edge_attr = edge_weights.reshape(-1, 1)

    # Convert to torch tensors
    x = torch.tensor(node_features, dtype=torch.float)
    node_type = torch.tensor(node_types, dtype=torch.long)
    edge_index_torch = torch.tensor(edge_index, dtype=torch.long)
    edge_attr_torch = torch.tensor(edge_attr, dtype=torch.float)
    target = torch.zeros(m, dtype=torch.float)  # For feasible solutions, ReLU(Ax - b) is zero

    # Create Data object
    data = Data(
        x=x,
        edge_index=edge_index_torch,
        edge_attr=edge_attr_torch,
        y=target,
        node_type=node_type
    )
    data_list.append(data)

print("Generating infeasible samples...")
for _ in tqdm(range(num_infeasible_samples)):
    x_combined = generate_infeasible_sample()
    # Compute ReLU(Ax - b)
    Ax = A @ x_combined
    Ax_minus_b = Ax - b_vector
    relu_Ax_minus_b = np.maximum(Ax_minus_b, 0)  # Shape: (m,)

    # Prepare node features and types
    variable_node_features = x_combined.reshape(-1, 1)  # Shape: (n, 1)
    variable_node_type = variable_types  # Shape: (n,)
    constraint_node_features = b_vector.reshape(-1, 1)  # Shape: (m, 1)
    constraint_node_type = np.full(m, 2)  # Constraint node type
    node_features = np.concatenate([variable_node_features, constraint_node_features], axis=0)  # Shape: (n + m, 1)
    node_types = np.concatenate([variable_node_type, constraint_node_type], axis=0)  # Shape: (n + m,)

    # Edge indices and attributes
    source_nodes = col_indices
    target_nodes = row_indices + n  # Shift constraint node indices
    edge_index = np.stack([source_nodes, target_nodes], axis=0)
    edge_attr = edge_weights.reshape(-1, 1)

    # Convert to torch tensors
    x = torch.tensor(node_features, dtype=torch.float)
    node_type = torch.tensor(node_types, dtype=torch.long)
    edge_index_torch = torch.tensor(edge_index, dtype=torch.long)
    edge_attr_torch = torch.tensor(edge_attr, dtype=torch.float)
    target = torch.tensor(relu_Ax_minus_b, dtype=torch.float)  # Non-zero for infeasible solutions

    # Create Data object
    data = Data(
        x=x,
        edge_index=edge_index_torch,
        edge_attr=edge_attr_torch,
        y=target,
        node_type=node_type
    )
    data_list.append(data)

# Shuffle the dataset
random.shuffle(data_list)

# Split into training and validation sets
train_ratio = 0.8
train_size = int(len(data_list) * train_ratio)
train_data_list = data_list[:train_size]
val_data_list = data_list[train_size:]

# Create DataLoaders
batch_size = 64  # Adjust based on your memory capacity
train_loader = DataLoader(train_data_list, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data_list, batch_size=batch_size, shuffle=False)

# Define the Message Passing GNN
class ConstraintGNN(MessagePassing):
    def __init__(self, node_feature_size, edge_feature_size, hidden_size):
        super(ConstraintGNN, self).__init__(aggr='mean')
        self.message_mlp = nn.Sequential(
            nn.Linear(node_feature_size + edge_feature_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size)
        )
        self.update_mlp = nn.Sequential(
            nn.Linear(node_feature_size + hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, node_feature_size)
        )

    def forward(self, x, edge_index, edge_attr):
        return self.propagate(edge_index, x=x, edge_attr=edge_attr)

    def message(self, x_j, edge_attr):
        m = torch.cat([x_j, edge_attr], dim=1)
        m = self.message_mlp(m)
        return m

    def update(self, aggr_out, x):
        u = torch.cat([x, aggr_out], dim=1)
        u = self.update_mlp(u)
        return u

# Define the overall GNN model
class ConstraintGNNModel(nn.Module):
    def __init__(self, node_feature_size, edge_feature_size, hidden_size, num_layers):
        super(ConstraintGNNModel, self).__init__()
        self.layers = nn.ModuleList([
            ConstraintGNN(node_feature_size, edge_feature_size, hidden_size)
            for _ in range(num_layers)
        ])
        self.fc = nn.Linear(node_feature_size, 1)

    def forward(self, data):
        x, edge_index, edge_attr, node_type = (
            data.x, data.edge_index, data.edge_attr, data.node_type
        )
        for layer in self.layers:
            x = layer(x, edge_index, edge_attr)
        # Extract features of constraint nodes
        constraint_node_mask = (node_type == 2)
        constraint_node_features = x[constraint_node_mask]
        out = self.fc(constraint_node_features).squeeze()
        out = F.relu(out)
        return out

# Model parameters
node_feature_size = 1  # One feature per node
edge_feature_size = 1  # Edge weight
hidden_size = 32
num_layers = 3

# Instantiate the model
model = ConstraintGNNModel(node_feature_size, edge_feature_size, hidden_size, num_layers)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training loop
num_epochs = 10  # Adjust the number of epochs as needed

print("Starting training...")
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        optimizer.zero_grad()
        # Forward pass
        output = model(batch)
        target = batch.y.to(output.device)

        # Check if output and target sizes match
        if output.shape != target.shape:
            print(f"Output size: {output.shape}, Target size: {target.shape}")
            continue  # Skip this batch or handle the mismatch appropriately

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_loss:.6f}")

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            output = model(batch)
            target = batch.y.to(output.device)
            if output.shape != target.shape:
                print(f"Validation Output size: {output.shape}, Target size: {target.shape}")
                continue
            loss = criterion(output, target)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {avg_val_loss:.6f}")

print("Training completed.")

# Evaluation on test data
num_test_samples = 500  # Adjust as needed
ground_truth_labels = []
predicted_labels = []

print("Evaluating model on test solutions...")
for _ in tqdm(range(num_test_samples)):
    # Randomly decide whether to use a feasible or infeasible sample
    if np.random.rand() < 0.5 and len(feasible_solutions) > 0:
        # Use a feasible sample
        x_combined = random.choice(feasible_solutions)
        is_feasible = True
    else:
        # Generate an infeasible sample
        x_combined = generate_infeasible_sample()
        is_feasible = False

    ground_truth_labels.append(int(is_feasible))

    # Prepare data and predict feasibility
    variable_node_features = x_combined.reshape(-1, 1)
    variable_node_type = variable_types
    constraint_node_features = b_vector.reshape(-1, 1)
    constraint_node_type = np.full(m, 2)
    node_features = np.concatenate([variable_node_features, constraint_node_features], axis=0)
    node_types = np.concatenate([variable_node_type, constraint_node_type], axis=0)
    source_nodes = col_indices
    target_nodes = row_indices + n
    edge_index = np.stack([source_nodes, target_nodes], axis=0)
    edge_attr = edge_weights.reshape(-1, 1)

    x = torch.tensor(node_features, dtype=torch.float)
    node_type = torch.tensor(node_types, dtype=torch.long)
    edge_index_torch = torch.tensor(edge_index, dtype=torch.long)
    edge_attr_torch = torch.tensor(edge_attr, dtype=torch.float)

    data = Data(
        x=x,
        edge_index=edge_index_torch,
        edge_attr=edge_attr_torch,
        node_type=node_type
    )

    model.eval()
    with torch.no_grad():
        output = model(data)
    max_output = torch.max(output).item()
    predicted_is_feasible = max_output <= 1e-4  # Adjust epsilon as needed
    predicted_labels.append(int(predicted_is_feasible))

# Compute accuracy metrics
accuracy = accuracy_score(ground_truth_labels, predicted_labels)
precision = precision_score(ground_truth_labels, predicted_labels)
recall = recall_score(ground_truth_labels, predicted_labels)
f1 = f1_score(ground_truth_labels, predicted_labels)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")


Number of solutions found: 1000
Preparing feasible samples...


100%|██████████| 1000/1000 [00:00<00:00, 18912.26it/s]


Generating infeasible samples...


100%|██████████| 1000/1000 [00:00<00:00, 3156.60it/s]


Starting training...


...

Epoch 9/10, Training Loss: 0.000394
Epoch 9/10, Validation Loss: 0.000244


Epoch 10/10: 100%|██████████| 25/25 [00:00<00:00, 63.46it/s]


Epoch 10/10, Training Loss: 0.000198
Epoch 10/10, Validation Loss: 0.000116
Training completed.
Evaluating model on test solutions...


100%|██████████| 500/500 [00:00<00:00, 559.98it/s]

Test Accuracy: 1.0000
Test Precision: 1.0000
Test Recall: 1.0000
Test F1 Score: 1.0000
